# Credit Approval Analysis

Here you can find the description of all the tasks. Same info and more can be found on this Project [Page](https://cms.master.school/final-project-jan23)

First review the project on this page and if you decide to work on this dataset follow the steps mentioned on above Project page


## The story behind The Data

Credit score cards are a common risk control method in the financial industry. 
The industry uses personal information and data submitted by credit card applicants to predict the probability of future defaults and credit card borrowings. 
The bank is able to decide whether to issue a credit card to the applicant. 
Credit scores can objectively quantify the magnitude of risk. 
As such, in this scenario, our bank HereIsTheMoney would like to find out what are the main factors that allow them to approve or deny a credit card to a client, and ultimately have an automated logic to identify these cases before they come to light, thus saving costs as well as increasing its reputation at the same time.

**A full ERD can be found [here](https://dbdiagram.io/d/638ce1b7bae3ed7c45449f75)**

# Data Description
In this task, few datasets are provided:

1. **`application_record.csv` -** this file contains appliers personal information, which you could use as features for data enrichment, in order to get better insights on the business use-case. The file contains ~438K records, and 18 columns.
The relevant columns this file has are:

	- `ID` - client number. Unique identifier for the customer holding the account
    - `AMT_INCOME_TOTAL` - annual income
    - `NAME_INCOME_TYPE` - income category
    - `NAME_EDUCATION_TYPE` - education level
    - `NAME_FAMILY_STATUS` - martial status
    - `NAME_HOUSING_TYPE` - way of living
    - `DAYS_BIRTH` - Count backwards from current day (0), -1 means yesterday
    - `DAYS_EMPLOYED` - Start date of employment; Count backwards from current day(0). If positive, person unemployed.
    - `OCCUPATION_TYPE` - Occupation
    
2. **`credit_record.csv` -** contains records users transactions of credit card. This dataset contains ~200K transactions, and has 3 columns:

    - `ID` - Client number
    - `MONTHS_BALANCE` - Record month; The month of the extracted data is the starting point, backwards, 0 is the current month, -1 is the previous month, and so on
    - `STATUS` - Status; A mapping column between a numerical value, and a range of time past due. Specifically - 0: 1-29 days past due; 1: 30-59 days past due; 2: 60-89 days overdue; 3: 90-119 days overdue; 4: 120-149 days overdue; 5: Overdue or bad debts, write-offs for more than 150 days; C: paid off that month X: No loan for the month

3. **`enriched_demographics_client.csv`** - this dataset contains some additional demographic information about each client. Specifically, it contains:
 
     - `ID` - Client number
     - `CODE_GENDER` - M (male) / F (female) of the client’s identity
     - `CNT_CHILDREN` - number of children
     - `CNT_FAM_MEMBERS` - Family size

4. ****`enriched_flags_client.csv`**** - this dataset contains some additional flags about each client. Specifically, it contains:
    - `ID` - Client number
    - `FLAG_MOBIL` - Is there a mobile phone
    - `FLAG_WORK_PHONE` - Is there a work phone
    - `FLAG_PHONE` - Is there a phone
    - `FLAG_EMAIL` - Is there an email
    - `FLAG_OWN_CAR` - Y (yes) / N (no) flag indicates if the client has a car
    - `FLAG_OWN_REALTY` - Y(yes) / N (no) indicates if the client has a property

# SQL tasks

1. How many clients that have more then 2 children does the bank have?
2. What is the average family size per each occupation type (based on fam_members)?
3. Out of the men clients, what is the count of transaction that were made, per each fam_member value (hint: treat fam_members as a dimension, not a metric)?
4. The bank is interested in knowing per each “status” (see above), who are the client IDs with top-5 highest AMT_INCOME_TOTAL, as well as the bottom-5 lowest AMT_INCOME_TOTAL. Please help him find that.
5. Let’s define a new column called “has_car_and_realty”. The bank thinks these people are affluent compared to others. Its logic is as follows:
- If flag_own_car = 1 and flag_own_realty = 1 ⇒ 1
- else ⇒ 0
Once we have this column, the bank would like to know per each has_car_and_realty category:

    1. how many unique clients do we have, 
    2. as well as how many transactions each category (1, 0) have had, 
    3. and what is the ratio between the two: unique_clients / num_transactions

Counting the number of total customers:

In [16]:
SELECT DISTINCT COUNT(ID) 
FROM enriched_demographics_client

count
0  438557

**Question** 1.** How many clients that have more then 2 children does the bank have?**

In [17]:
SELECT DISTINCT COUNT(ID) 
FROM enriched_demographics_client
WHERE cnt_children > 2

count
0   6075

Answer: 6075 clients with more then 2 children.

confirming by counting the rest of the customers to complete to total amount:

In [18]:
SELECT DISTINCT COUNT(ID) 
FROM enriched_demographics_client
WHERE cnt_children <= 2

count
0  432482

6075 clients with more then 2 children, 432482 with 2 or less, both combined complete the total number of customers.

Question **2.** What is the average family size per each occupation type (based on fam_members)?

In [20]:
SELECT OCCUPATION_TYPE,
       ROUND(AVG(CNT_FAM_MEMBERS), 2) AS average_fam_size    
FROM enriched_demographics_client
JOIN application_record
USING (ID)
WHERE OCCUPATION_TYPE IS NOT NULL
GROUP BY OCCUPATION_TYPE
ORDER BY 2 desc

occupation_type  average_fam_size
0                 Drivers              2.41
1              Core staff              2.34
2                Laborers              2.32
3           Realty agents              2.32
4                Managers              2.31
5             Sales staff              2.27
6             Accountants              2.26
7      Low-skill Laborers              2.26
8             Secretaries              2.25
9   Private service staff              2.25
10          Cooking staff              2.25
11  High skill tech staff              2.24
12         Medicine staff              2.22
13         Security staff              2.22
14   Waiters/barmen staff              2.17
15         Cleaning staff              2.07
16               HR staff              2.02
17               IT staff              1.97

Answer: average family size by occupation, rounded and ordered by the family sizes from larger to smaller.

Question **3.** Out of the men clients, what is the count of transactions that were made, per each fam_member value?

In [21]:
SELECT  CNT_FAM_MEMBERS AS num_family_members,
        COUNT(credit_record.ID) AS transaction_count
FROM enriched_demographics_client
JOIN credit_record
USING(ID)
WHERE CODE_GENDER = 'M'
GROUP BY CNT_FAM_MEMBERS

num_family_members  transaction_count
0                   1               6502
1                   2              27014
2                   3              11253
3                   4               6156
4                   5                806
5                   6                 48
6                   7                 31
7                  15                 23

Answer: out of men only, count of transactions per number of familiy members, orders by the size of the family from smaller to larger. Assuming household size is inclusive, men who live by themselves have 6.5K transactions total for example, and it looks like those who live with one more household member as a family of two, are the group with the most transactions compared to others.

Question **4.** The bank is interested in knowing per each “status”, who are the client IDs with top-5 highest AMT_INCOME_TOTAL, as well as the bottom-5 lowest AMT_INCOME_TOTAL. Please help him find that.

In [22]:
SELECT  id, 
        income,
        status, 
        (CASE WHEN top_5 <=5 THEN 'highest earning' 
              WHEN bottom_5 <=5 THEN 'lowest earning' END) AS client_income
        
FROM (
        SELECT  
        amt_income_total as income, 
        id,
        status,
        RANK() over (partition by status order by amt_income_total desc) top_5,
        RANK() over (partition by status order by amt_income_total) bottom_5
        FROM application_record
        JOIN credit_record
        USING (ID) 
) AS ranks
WHERE top_5 <= 5 or bottom_5 <= 5
order by status, top_5


id     income status    client_income
0    5143232  1575000.0      0  highest earning
1    5143236  1575000.0      0  highest earning
2    5143232  1575000.0      0  highest earning
3    5143234  1575000.0      0  highest earning
4    5143237  1575000.0      0  highest earning
..       ...        ...    ...              ...
136  5045657    30150.0      X   lowest earning
137  5135925    29250.0      X   lowest earning
138  5135925    29250.0      X   lowest earning
139  5135925    29250.0      X   lowest earning
140  5135925    29250.0      X   lowest earning

[141 rows x 4 columns]

Answer: ranking of top 5 and bottom 5 by annual income per loan status group, considering that there are some clients that are ranked equally, due to same income- there are more than 5*8*2 (rank, status group, top or bottom) results. Results are ordered by status groups, and each group shows highest ranked, then lowest ranked. 

Results show larger count of records due to clients with identical income.

Question **5.** Let’s define a new column called “has_car_and_realty”. The bank thinks these people are affluent compared to others. Its logic is as follows:
If flag_own_car = 1 and flag_own_realty = 1 ⇒ 1
else ⇒ 0
Once we have this column, the bank would like to know per each has_car_and_realty category:

    1. how many unique clients do we have, 
    2. as well as how many transactions each category (1, 0) have had, 
    3. what is the ratio between the two: u*nique_clients / num_transactions

Quick overview of customers overall

In [23]:
select count(distinct id), FLAG_OWN_REALTY, FLAG_OWN_car
from enriched_flags_client
group by 2, 3

count flag_own_realty flag_own_car
0   84340               N            N
1   50142               N            Y
2  191110               Y            N
3  112951               Y            Y

Ratio between clients who own both house and car and the rest of the client population:

In [24]:
SELECT CASE WHEN FLAG_OWN_CAR = 'Y' AND FLAG_OWN_REALTY = 'Y' THEN 1 ELSE 0 END AS has_car_and_reality, 
        COUNT(DISTINCT id) AS unique_clients, 
        ROUND(1.0*COUNT(DISTINCT id)/(SELECT COUNT(*) FROM enriched_flags_client),2) AS ratio
FROM enriched_flags_client
GROUP BY has_car_and_reality;

has_car_and_reality  unique_clients  ratio
0                    0          325572   0.74
1                    1          112951   0.26

~113K of applied customers are favorites- own both house and car (~26%)

Quick overview of these customers with credit records:

In [25]:
select count(distinct id),
        FLAG_OWN_REALTY, 
        FLAG_OWN_car
from credit_record
join enriched_flags_client
using(id)
group by 2, 3 

count flag_own_realty flag_own_car
0   6530               N            N
1   4270               N            Y
2  13566               Y            N
3   8279               Y            Y

8279 of the loan customers own both house and car (~25.5%), similar to general customer population.

Customer distribution within status (interactive query by changing the status to see different results):

In [26]:
select * from
   ( select  count(distinct id), FLAG_OWN_REALTY, FLAG_OWN_car
from 
  credit_record
   join
  enriched_flags_client
  using(id)
    where status = 'X'
   group by 2,3) as status_group

count flag_own_realty flag_own_car
0   1993               N            N
1   1373               N            Y
2   4159               Y            N
3   2549               Y            Y

Conducted the query for each status group, results include some customers who are in more than one status. Found very similar representation of these customers who own both, in percentage, within status group 0, X and C, and lower representation in groups 1 and 2 (~21.5&). Groups 3, 4 and 5 are too small to derive conclusions. 
**Customers who own both a house and car do seem to be generally much more reliable than the rest when it comes to payments (although may still be favorites due to having assets with with they can guarantee for payback on the loan).

Question **5.** Let’s define a new column called “has_car_and_realty”.

In [27]:
SELECT  (CASE WHEN FLAG_OWN_CAR = 'Y' 
        AND FLAG_OWN_REALTY = 'Y' THEN 1 
        ELSE 0 END) as has_car_and_realty
FROM    enriched_flags_client

has_car_and_realty
0                        0
1                        0
2                        0
3                        0
4                        0
...                    ...
438552                   0
438553                   0
438554                   0
438555                   0
438556                   0

[438557 rows x 1 columns]

Column was created for all rows.

 1. how many unique clients do we have (this query refers to the entire data of clients)

438510 clients total (all bank customers).

In Transactions:
1. how many unique clients do we have
2. how many transactions each category (1, 0) have had
3. what is the ratio between the two: unique_clients / num_transactions

In [29]:
select *, customer_count*100/transaction_count as ratio from 
(select count(distinct ID) as customer_count, has_car_and_realty, count(id) as transaction_count from(
    SELECT  id, (CASE WHEN FLAG_OWN_CAR = 'Y' 
        AND FLAG_OWN_REALTY = 'Y' THEN 1 
        ELSE 0 END) as has_car_and_realty
    FROM    enriched_flags_client
    join credit_record
     USING (ID)) gg
group by has_car_and_realty) car_home

customer_count  has_car_and_realty  transaction_count  ratio
0           24366                   0             116004     21
1            8279                   1              39512     20

Answer: Clients in credit records- 8279 total clients who own both a house and car and have 39.5K transaction records, the ratio between the clients and transactions is 20%. 24.4K are the rest of the customers, and have 116K transaction records, 21% ratio, which is similar ratios between the groups.

So how many customers who applied but do not have existing credit are in each housing arrangement?

In [30]:
select count(distinct id), FLAG_OWN_CAR, FLAG_OWN_REALTY
from enriched_flags_client
where id not in (
select id
from credit_record
) 
group by 2, 3
order by 1

count flag_own_car flag_own_realty
0   45872            Y               N
1   77810            N               N
2  104672            Y               Y
3  177544            N               Y

In [31]:
select count( id)
from application_record
where id not in (
select id from credit_record)

count
0  405912

405.9K customers who applied do not have an existing record in credit.

Versus clients who applied and have an existing loan:

In [32]:
select count(distinct id), FLAG_OWN_CAR, FLAG_OWN_REALTY
from enriched_flags_client
where id in (
select id 
from credit_record
) 
group by 2, 3
order by 1

count flag_own_car flag_own_realty
0   4270            Y               N
1   6530            N               N
2   8279            Y               Y
3  13566            N               Y

Customers who own both a house and cars but do not have a loan record represent about 25% of the population, just like their weighted average in the customer population. These customers also are about 25% of the approved credit population so represent so they do not seem to be more likely approved or denied.

In [34]:
select count(distinct id)
from application_record
where id in (
select id from credit_record)

count
0  32645

32.5K clients who applied have records of existing/previous loan.

How many customers of each housing arrangement have credit record in a completed loan status?

In [35]:
select count(distinct id), FLAG_OWN_CAR, FLAG_OWN_REALTY
from enriched_flags_client
where id in (
select id 
from credit_record
where status = 'C'
) 
group by 2, 3
order by 1



count flag_own_car flag_own_realty
0   2046            Y               N
1   3152            N               N
2   3868            Y               Y
3   6401            N               Y

In [36]:
select count(id), occupation_type
from application_record
join credit_record
using (id)
where status = 'C'
group by 2

count        occupation_type
0    2392            Accountants
1     955         Cleaning staff
2     976          Cooking staff
3    6522             Core staff
4    3977                Drivers
5    2635  High skill tech staff
6     125               HR staff
7     128               IT staff
8   11296               Laborers
9     403     Low-skill Laborers
10   5592               Managers
11   2260         Medicine staff
12    488  Private service staff
13     91          Realty agents
14   5909            Sales staff
15    307            Secretaries
16   1085         Security staff
17    211   Waiters/barmen staff
18  20703                   None

These customers still represent the same portion of the population- they are not more likely than the rest to complete the loan.

How many customers in total live with their parents and what are their demographics?

In [37]:
select count(*)
from application_record
where NAME_HOUSING_TYPE = 'With parents'

count
0  19077

People living with their parents represent 4.3% of total customers. There are 19.1K customers total. Below are some of their demographics.

In [46]:
select  count(distinct id) as num_customers, 
        NAME_HOUSING_TYPE as living_arrangement, 
        round(avg(AMT_INCOME_TOTAL)) as average_income, 
        round(avg(DAYS_BIRTH / -365), 2) as age,
        OCCUPATION_TYPE,
        CNT_CHILDREN,
        case when avg(DAYS_EMPLOYED) > 0 then 0
        else round(avg(DAYS_EMPLOYED / -365)) end as years_employed
from application_record
join enriched_demographics_client
using(id)
where NAME_HOUSING_TYPE = 'With parents' 
group by 2, 5, 6
order by num_customers desc

num_customers living_arrangement  ...  cnt_children  years_employed
0            2430       With parents  ...             0               5
1            2296       With parents  ...             0               0
2            1625       With parents  ...             0               5
3            1581       With parents  ...             0               4
4            1158       With parents  ...             1               5
..            ...                ...  ...           ...             ...
63              2       With parents  ...             3              11
64              2       With parents  ...             2               5
65              2       With parents  ...             2               4
66              2       With parents  ...             4              13
67              2       With parents  ...             4               4

[68 rows x 7 columns]

In [40]:
select count(distinct id)
from credit_record 

count
0  41628

A total of 41.6K customers have records of credit.

In [41]:
select num_customers from 
(select count(distinct id) as num_customers, NAME_HOUSING_TYPE 
from application_record
join credit_record
using(id)
group by 2) parents
where NAME_HOUSING_TYPE = 'With parents' 

num_customers
0           1594

1.6K live with their parents.

In [43]:
select status, num_customers from 
(select count(distinct id) as num_customers,NAME_HOUSING_TYPE, status
from application_record
join credit_record
using(id)
group by 2, status) hh
where NAME_HOUSING_TYPE = 'With parents' 

status  num_customers
0      0           1126
1      1             88
2      2             10
3      3              2
4      4              1
5      5              5
6      C            776
7      X            468

7250, 42% completed status (same as general population- no further analysis to be made)
here are 2476 customers in credit records who live with their parents. They represent 6% of the credit customers. The majority (45.5%) are 1 payment late- group 0. 31.5% completed, 19% no loan, 3.5% are 2 payments behind, and a small perecentage is further behind on payments. When compared to the general population of credit loans, no need in further analysis seem to arise at this point.

In [34]:
select count(id), status
from credit_record
group by status

In [35]:
select count(id)
from credit_record

210K transactions total. The major status is 1 payment late- 24K customers, followed by 15.5K completed, and 10K no loan record this month (possibly with some customer overlap if several transactions exist per customer). About 54K of the customers with credit records did not apply on the currant period:

In [36]:
select count(id)
from credit_record
left join application_record
using(id)
WHERE application_record.id IS NULL;

32.6K out of 41.6K of the customers with credit record also applied for credit, they have 155.5K existing records between them out of the 210K total. That leaves 9K customers with existing credit records who did not apply for another credit, they have about 54.5K existing transaction records between them.

In [37]:
SELECT credit_record.*
FROM credit_record
LEFT JOIN application_record
ON credit_record.id = application_record.id
WHERE application_record.id IS NULL;

Summary statistics on months in records of credit:

In [38]:
select count(ID) as total_records, round(avg(MONTHS_BALANCE*-1), 1) as average_months, status
from credit_record
group by status
order by 1 desc

List of 22 customers for automatic decline on application based on existing loan and cusomer employment status: severely late on payments; loan is recent and is mostly unpaid; customer unemployed:

In [39]:
select distinct id from application_record
where id in
(select  id
from credit_record
where status BETWEEN '2' AND '5'
and MONTHS_BALANCE >-12)
and DAYS_EMPLOYED >=0

## Tableau Tasks

This Tableau task lets you be creative with the way you build your dashboard and only gives guidance on the types of visualisations you should use.

Create a dashboard that will contain at least 5 out of the 8 requirements defined below:

1. **At least 2 KPIs** as Big Ass Numbers. Choose the metrics on your own that will fit with the rest of your analysis. Examples of metrics:
    - Number of bank clients
    - % of clients with higher education
    - etc.
2. **A Bar Chart**, for example, comparing the number of clients in each client segment (custom defined as a combination of demographic characteristics).
3. **A Stacked Bar Chart with Quick Table Calculation**, for example, for each Family Status comparing the ratio of car owners and non-car owners.
4. **A Highlight Table (Heatmap)**, for example, between education and income level (can be custom-defined based on Amt Income Total), where each cell corresponds to absolute or relative numbers of clients in each combination. 
5. **A Circle View** with three different values on X axis, Y axis, and Size.
6. **A Pareto Analysis** based on a custom-defined **client segment** to answer a question: Which client segments make up 80% of all client base? 
    - Segment can include values from the demographic and/or record information about the client. For example, Education | Income | Marital status | Owning a car | No children.
7. **At least 1 parameter** that allows dashboard user to interact with the dashboard.
8. At least **1 more visualization of your choice** adding value to your analysis.

Tableau 
https://public.tableau.com/views/CreditRisks/Credit?:language=en-US&:display_count=n&:origin=viz_share_link

After exploring the datasets and these above questions, I conducted a large part of the data exploration and cleaning in Tableau, mainly:
* Joining the datasets 
* Changing some of the column names to make them more clear, such as AMT_INCOME_TOTAL to Annual Household Income 
* Detection of outliers, such as exessively high annual income in a large amount of rows, which I chose to treat as an entry error 
* Converted the column DAYS_BIRTH into appropriate age in whole years and the column MONTHS_BALANCE into appropriate number of months by creating calculated fields
* Converting impossible numbers- 1000 years
* Created bins for age, by age groups
* Converted some of the data types of the columns
* Creating new column calculated from other columns-
-"own house and car" to represent the customers who own both in comprison with the rest of the customers
-date column of year customer began employment, created from the time provided for months of employment, starts from current year
-percent on record represents the ratio between the time the customer had the loan on record and the time customer has been employed
-percent total transactions as a ratio of bank customers
* Clarifying status groups of loan and the gender column by aliasing with new names instead of numbers and letters
* Hiding irrelevant columns 
* Rounding some of the numerical columns from excessive decimals to the appropriate number of decimals of whole numbers
* Translated null values from no matching records between credit table and application table into applicants who were denied
* Correcting incorrect information derived from viewing two related columns- time of employment in a record where the customer is unemployed

Questions explored:
1. How many customers in total does the bank have
2. What is the median annual household income of customers
3. What is the overall ratio between customers applying and customers approved? what percentage of the customer population is denied?
4. How many months on average does it take for customers to complete the loan
5. what are the demographics of customers who live with their parents? are they more likely to be denied? are they more reliable in paying the loan?
6. What is the distribution of applicants based on home, vehicle ownership
7. Who are the majority and overall distribution of the customers in terms of ocuupations
8. Who are the customers with the most loans based on occupations, does it resemble the bank population?
9. What are the relatively stable customers based on time employed per occupation
10. What are the differences between females and males when it comes to loan payment status and months on record
11. Who are the most risk customers: severly behind on payment, are unemployed and do not own any car or house
12. What are some differences between age groups when it comes to customers who are severly late on their payments

In the first dashboard we have main KPIs and some critical demographic data about the customers, including occupation and ownership of home and car.

![Screen Shot 2023-02-15 at 15.30.25](Screen%20Shot%202023-02-15%20at%2015.30.25.png)


In the second dashboard, deeper dive into some insights related to the occupations within credit records.

![dash2](dash2.png)


In the third dashboard, warning insights related to risk of unpaid loans.

![dash3](dash3.png)


Challenges: Domain knowledge and understanding of the meta data is a challenge when trying to understand what a record means, what a staus means and also what serves the interest of the bank better.

Data Enrichment:
- There is certainly a major type of data that is missing and could have made the analysis more complete- this is the amount of the money loaned. 
- The other amount missing is the value of their car or house, as well as how much they own vs how much they owe on these.
- Another missing field or data that could be calculated into a field is credit score or total debt-to-income ratio. 
- The status of the borrowers is only known for the month the data was pulled, which limits the analysis to 5 months back, meaning: if a customer is at status "5" on their loan, we know they have also been late on payments during the past 5 months. However, if a customer is currently in status "1" for example, that does not eliminate a possibility that in the past the customer was in status 5.
- There is not enough contact information such as last name and address, which could have been used to estimate and analyze applications by loan history from household members.
- History of payments, number of missed payments
- interest rate or fees 

More exploration, cleaning and vizualisation in Python:

_For_ deep dive analysis and EDA follow the guidelines on project [Page](https://cms.master.school/final-project-jan23)

In [40]:
# Let's see how we can easily query the data
# Start working here if you choose to use Python for this part of the project
# Libraries
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd   
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

In [41]:
# Load data first and see its "head"
application_record_df = pd.read_csv("application_record.csv")
credit_record_df = pd.read_csv("credit_record.csv")
enriched_demographics_client_df = pd.read_csv("enriched_demographics_client.csv")
enriched_flags_client_df = pd.read_csv("enriched_flags_client.csv")

In [42]:
string = 'abcderbct'
print(string.partition('bc'))

In [43]:
application_record_df['OCCUPATION_TYPE'].value_counts()

In [44]:
co = credit_record_df['MONTHS_BALANCE']
absco = round(co.abs().corr(enriched_demographics_client_df['CNT_CHILDREN']), 5)
print(absco)

In [45]:
correl = credit_record_df['MONTHS_BALANCE']
absol = correl.abs().corr(enriched_demographics_client_df['CNT_FAM_MEMBERS'])
print(round(absol, 5))


make copy, fillna

In [ ]:
score_df['OCCUPATION_TYPE'] = score_df['OCCUPATION_TYPE'].fillna('Unknown')

In [ ]:
# Replace all occurrences of 'null' with 'Unknown'

all_clients_df['OCCUPATION_TYPE'] = all_clients_df['OCCUPATION_TYPE'].fillna('Unknown')

In [ ]:
sns.kdeplot(data=score_df, x='TRANSACTION_COUNT', y='AMT_INCOME_TOTAL', cmap="Oranges", fill=True)
plt.xlabel('Count of Transactions')
plt.ylabel('Total Income')
plt.ylim(0, 300000)
yticks = np.arange(0, 300001, 50000)
plt.gca().set_yticks(yticks)
plt.gca().set_yticklabels(yticks/1000)
plt.show()

summary statistics on numerical columns in applications

In [18]:
application_record_df.describe()[["DAYS_EMPLOYED", "AMT_INCOME_TOTAL"]].round()

total applications column info and missing records

In [19]:
application_record_df.info()

In [20]:
application_record_df.isna().sum()

In [21]:
appfilled= application_record_df.fillna("Missing")
appfilled.isna().sum()

In [22]:
credit_record_df.isna().any()

In [29]:
group = credit_record_df.groupby("STATUS")["ID"].count()
group.plot(kind= "bar", rot=0)
plt.xlabel('Status Group')
plt.ylabel('Loan Amount', rotation=0)
plt.legend(loc=2)
plt.title('Count of loans by status')
plt.show()

In [24]:
occup_income= application_record_df.groupby("OCCUPATION_TYPE")["AMT_INCOME_TOTAL"].mean().round() 
print(occup_income)

In [25]:
occup_income_sorted = occup_income.sort_values()
occup_income_sorted

In [26]:
occup_income.plot(kind= "bar", title= "avg income by occupation", x="OCCUPATION_TYPE", y="AMT_INCOME_TOTAL", rot=70)
plt.show

In [27]:
enriched_demographics_client_df[enriched_demographics_client_df["CODE_GENDER"]== "F"]["CNT_FAM_MEMBERS"].hist()
enriched_demographics_client_df[enriched_demographics_client_df["CODE_GENDER"]== "M"]["CNT_FAM_MEMBERS"].hist()
plt.legend("F", "M")
plt.show()


In [32]:
credit_record_df[credit_record_df["STATUS"]== "C"]["MONTHS_BALANCE"].hist()
credit_record_df[credit_record_df["STATUS"]== "X"]["MONTHS_BALANCE"].hist()
plt.legend("C", "5")
plt.show()

In [ ]:
print(taxi_own_veh['fuel_type'].value_counts())

In [2]:
num=[23, 45, 22, 89, 78, 90]
num[1:4]=10,

In [ ]:
airline_totals_sorted = airline_totals.sort_values("bumps_per_10k", ascending=False)

In [30]:
credit_record_df['STATUS'].value_counts()

In [89]:
application_record_df.head(3)

In [329]:
credit_record_df.head(3)

In [90]:
enriched_flags_client_df.head(3)

In [91]:
enriched_demographics_client_df.head(3)

In [92]:
# how many unique ID in application record?
len(set(application_record_df['ID']))

In [93]:
# how many unique ID in credit record?
len(set(credit_record_df['ID']))

In [94]:
# how many IDs do two tables share?
len(set(application_record_df['ID']).intersection(set(credit_record_df['ID'])))

In [332]:
# One can think of merging the two DFs

df = pd.merge(application_record_df, credit_record_df, how='inner', on=['ID'])

In [333]:
# Let's see all columns after merge of two DFs
df.columns

In [335]:
df.info()

In [97]:
enriched_flags_client_df['FLAG_OWN_CAR'].value_counts().plot(kind='pie', explode=(0.1,0), shadow=True, autopct='%1.1f%%')
# Seems that ~63% of the clients doesn't own a car...

In [336]:
enriched_flags_client_df['FLAG_OWN_REALTY'].value_counts().plot(kind='pie', explode=(0.1,0), shadow=True, autopct='%1.1f%%')


In [98]:
enriched_demographics_client_df['CNT_CHILDREN'].value_counts().plot(kind='pie', shadow=False, autopct='%1.1f%%')
centre_circle = plt.Circle((0, 0), 0.50, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
# ~70% doesn't have any children

In [ ]:
sns.set(font_scale=1.0)
fig, axes = plt.subplots(nrows=2,ncols=2)
fig.set_size_inches(10, 10)
sns.boxplot(data=combined_df,y="Age_in_years",orient="v",ax=axes[0][0])
sns.boxplot(data=combined_df,y="Age_in_years",x="OCCUPATION_TYPE",orient="v",ax=axes[0][1],plt.xticks(rotation=30))
sns.boxplot(data=combined_df,y="Age_in_years",x="NAME_HOUSING_TYPE",orient="v",ax=axes[1][0])
sns.boxplot(data=combined_df,y="Age_in_years",x="NAME_INCOME_TYPE",orient="v",ax=axes[1][1])
plt.xticks(rotation=30)
axes[0][0].set(ylabel='Age',title="Client age")
axes[0][1].set(xlabel='Occupation Status', ylabel='Age',title="Box Plot according to occupation")
axes[1][0].set(xlabel='Housing Status', ylabel='Age',title="Box Plot according to housing status")
axes[1][1].set(xlabel='Income Type', ylabel='Age',title="Box Plot according to the income type")


In [99]:
# Let's deep dive into the values
enriched_demographics_client_df['CNT_CHILDREN'].value_counts()

In [100]:
#counts and percentages of clients based on home ownership
counts = enriched_flags_client_df['FLAG_OWN_REALTY'].value_counts()
counts=counts.apply('{:,}'.format)
percs = enriched_flags_client_df['FLAG_OWN_REALTY'].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
pd.concat([counts,percs], axis=1, keys=['count', 'percentage'])


Summary Statistics short analysis:
Basic statistics- defined min, max, median, percentiles, and average of the numerical fields days employed and total income with .describe method. Checked counts of values of occupations,  found substantial amount of nulls in one field (occupation) and outliers in the income field. created counts of total customers in applications and in credit records, as well as within sub-groups of those, such as by occupation.

Data Cleaning
outliers-
nulls-
data types-
wrong value types-

Distribution Analysis

Cross-Correlation Analysis
correlation between months on balance and size of the family,

Raising Data Questions

Challenges: Domain knowledge and understanding of the data is a challenge when trying to understand what a record means, what a staus means and also what serves the interest of the bank better.
Data Enrichment:
There is certainly a major type of data that is missing and could have made the analysis more complete- this is the amount of the money loaned. 
The other amount missing is the value of their car or house, as well as how much they own vs how much they owe on these.
Another missing field or data that could be calculated into a field is credit score. 
The status of the borrowers is only known for the month the data was pulled, which limits the analysis to 5 months back, meaning: if a customer is at status "5" on their loan, we know they have also been late on payments during the past 5 months. However, if a customer is currently in status "1" for example, that does not eliminate a possibility that in the past the customer was in status 5.
There is not enough contact information such as last name and address, which could have been used to estimate and analyze applications by loan history from household members.
interest rate debt-to-income ratio number of missed payments

business recommendations

Summary of Work
